The old, simple way to initialize a parent class from a child class is to directly call the parent class's init method with the child instance

In [1]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value


class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBeseClass.__init__(self, 5)
        
#this approach works fine for basic class hierarchies but breaks in many cases.

If a class isaffected by multiple inheritance, calling the superclasses' ___init___ methods directly can lead to unpredictable behavior

In [2]:
class TimesTwo:
    def __init__(self):
        self.value *= 2


class PlusFive:
    def __init__(self):
        self.value += 5
        
#consturcting it prodeces a result that matches the parent class order
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [3]:
foo = OneWay(5)
print('First ordering value is (5 * 2) + 5 =', foo.value)

First ordering value is (5 * 2) + 5 = 15


In [4]:
#in different order
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

        
bar = AnotherWay(5)
print('Second ordering value is (5 * 2) + 5 =', bar.value)

Second ordering value is (5 * 2) + 5 = 15


Diamond inheritance happens when a subclass inherits from two separate classes that have the same superclass somewhere in the hierarchy. Diamond inheritance causes the common superclass's init method toe run multiple times, causing unexpected behavior.

In [5]:
class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7
        
        
class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9
        
        
class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)
        
        
foo = ThisWay(5)
print('Should be (5 * 7) + 9 = 44 but is', foo.value)
    

Should be (5 * 7) + 9 = 44 but is 14


### To solve these problems, Python has the super built-in function and standard method resolution order(MOR)

* super ensures that common superclasses in diamond hierarchies are run only once
* The MRO defines the ordering in which superclasses are initialized, following an algorithm called C3 linearization.

In [6]:
class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 7
        
        
class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__( value)
        self.value += 9

In [7]:
#Now, the top part of the diamond, MyBaseClass.__init++, is run only a single time. 
#The other parent classes are run in the order specified in the class statement

class GoodWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)
        

foo = GoodWay(5)
print('Should be 7 * (5 + 9) = 98 but is', foo.value)

Should be 7 * (5 + 9) = 98 but is 98


In [8]:
mro_str = '\n'.join(repr(cls) for cls in GoodWay.mro())
print(mro_str)

<class '__main__.GoodWay'>
<class '__main__.TimesSevenCorrect'>
<class '__main__.PlusNineCorrect'>
<class '__main__.MyBaseClass'>
<class 'object'>


In [11]:
class ExplictTrisect(MyBaseClass):
    def __init__(self, value):
        super(ExplictTrisect, self).__init__(value)
        self.value /= 3
        
        
class AutomaticTrisect(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value)
        self.value /= 3
        
        
class ImplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value /= 3
        

assert ExplictTrisect(9).value == 3
assert AutomaticTrisect(9).value == 3
assert ImplicitTrisect(9).value == 3

The only time you should provide parameters to super is in situations where you need to access the specific functionality of a superclass's implementation from a  child class